In [1]:
# Standard Libraries of Python
from decimal import Decimal, ROUND_HALF_UP, getcontext
getcontext().prec = 5

# Dependencies
import pandas as pd
import numpy as np
np.set_printoptions(precision=5)

# Libraries made for this Proyect
from database.clean_database import database
from data_analisys import data_functions

In [2]:
# Main Object

class Analisis:
    def __init__(self):
        self.df = pd.read_parquet('database/db.parquet')
        self.lottery = self.df.copy()
        self.lottery['Dates'] = self.lottery['Dates'].dt.year
        self.lottery = self.lottery.drop(columns=['Sorteo','Star_1','Star_2'])
        self.numbers = range(1,51)

    def __transformation_into_columns(self,row):
        for draw in range(1,6):
            if not np.isnan(self.new_lottery.loc[row.Dates,row[f'Nro{draw}']]):
                self.new_lottery.loc[row.Dates,row[f'Nro{draw}']] += 1
            else:
                self.new_lottery.loc[row.Dates,row[f'Nro{draw}']] = 1
    
    def __numbers_boolean(self):
        self.skips_winners_bool = pd.DataFrame(False, columns=[int(i) for i in self.numbers],
            index=range(len(self.df)))
        row_0 = pd.DataFrame(columns=[int(i) for i in self.numbers],
            index=[0]).fillna(True)
        for e in range(1,6):
            col_name = f"Nro{e}"
            self.skips_winners_bool = self.skips_winners_bool | (self.df[col_name].to_numpy()[:, None] == self.numbers)
        self.boolean_df = pd.concat([row_0, self.skips_winners_bool]).reset_index(drop=True)

    def __total_average_hits(self,is_star=False,aprox=False):
        divide = 2 if is_star else 5
        self.average = self.hits.apply(lambda hits: hits / len(self.df) / divide).iloc[0]
        if aprox:
            return Decimal(self.average.sum()) / Decimal(int(50)) + Decimal(0.001)
        else:
            return Decimal(self.average.sum()) / Decimal(int(50))

    def __natural_rotation(self,is_star=False,aprox=False):
        self.__total_average_hits(is_star=is_star,aprox=aprox)
        rotation = pd.DataFrame({'hits': self.hits.iloc[0],
            'average_numbers': self.average,
            'average': self.__total_average_hits(is_star=is_star,aprox=aprox),
            'hits_needed': self.m_hits(is_star=is_star,aprox=aprox)},
            index=self.numbers)
        rotation['difference'] = rotation['hits'] - rotation['hits_needed']
        return rotation
        
    def apply_transformation(self):
        self.new_lottery = pd.DataFrame(columns=[i for i in self.numbers],
            index=np.arange(self.lottery['Dates'].iloc[0],self.lottery['Dates'].iloc[-1]+1))
        self.lottery.apply(self.__transformation_into_columns,axis=1)
        self.new_lottery.fillna(0,inplace=True)

        self.hits = self.new_lottery.sum().to_frame().rename(columns={0: 'hits'}).T.astype('int32')
        self.mean = self.new_lottery.mean().to_frame().rename(columns={0: 'mean'}).T.astype('float32')
        self.median = self.new_lottery.median().to_frame().rename(columns={0: 'median'}).T.astype('float32')

    def m_hits(self,is_star=False,aprox=False):
        min_hits = self.__total_average_hits(is_star=is_star,aprox=aprox)
        return min_hits * Decimal(int(self.hits.iloc[0,0])) / Decimal(float(self.average.iat[0]))

    def get_natural_rotations(self,is_star=False):
        self.exact_rotation = self.__natural_rotation(is_star=is_star,aprox=False)
        self.aprox_rotation = self.__natural_rotation(is_star=is_star,aprox=True)

    def count_skips(self):
        self.__numbers_boolean()
        dicts = {draw: {key: [] for key in self.numbers} for draw in range(1,len(self.boolean_df))}
        for draw in range(1,len(self.boolean_df)):
            df = self.boolean_df.loc[:draw]
            counts = {key: 0 for key in self.numbers}
            for column in df:
                counter = 0
                for value in reversed(df[column]):
                    if not value:
                        counter += 1
                    else:
                        counts[column] = counter
                        break
            dicts[draw].update(counts)
        self.counts = pd.DataFrame.from_dict(dicts,orient='index')
    
    def numbers_clasification(self):
        best_numbers = self.aprox_rotation.loc[self.aprox_rotation['hits'] > self.aprox_rotation['hits_needed']].index.to_numpy()
        normal_numbers = self.exact_rotation.loc[(self.exact_rotation['hits'] > self.exact_rotation['hits_needed']) & ~(self.exact_rotation.index.isin(best_numbers))].index.to_numpy()
        rotation_info = {number: 0 for number in self.numbers}
        for number in best_numbers:
            rotation_info[number] = 2
        for number in normal_numbers:
            rotation_info[number] = 1

In [69]:
euromillones = Analisis()
euromillones.apply_transformation()
euromillones.get_natural_rotations()
euromillones.count_skips()

In [137]:
class Perro:
    def __init__(self) -> None:
        pass

    def __str__(self) -> str:
        self = 'El perro'

'Mira un alga marina'

In [111]:
euromillones.lottery.drop(columns='Dates')

,Nro1,Nro2,Nro3,Nro4,Nro5
0,16,29,32,36,41
1,7,13,39,47,50
2,14,18,19,31,37
3,4,7,33,37,39
4,15,24,28,44,47
...,...,...,...,...,...
1621,5,26,28,36,46
1622,10,14,16,19,29
1623,4,11,37,44,49
1624,17,21,23,35,37


In [104]:
objetivo = pd.DataFrame()
objetivo = pd.DataFrame(columns=euromillones.numbers)

objetivo.loc[1] = 1 if euromillones.boolean_df_especial.loc[1].where((euromillones.boolean_df_especial.loc[1], 0)) == False else 0

for i in range(2,len(euromillones.boolean_df_especial)):
    objetivo.loc[i] += 1 if euromillones.boolean_df_especial.loc[i].to_frame().T == False else 0
    

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [133]:
euromillones.boolean_df_especial.where(euromillones.boolean_df_especial==True,1).where(euromillones.boolean_df_especial==False,0).loc[1]

1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    0
17    1
18    1
19    1
20    1
21    1
22    1
23    1
24    1
25    1
26    1
27    1
28    1
29    0
30    1
31    1
32    0
33    1
34    1
35    1
36    0
37    1
38    1
39    1
40    1
41    0
42    1
43    1
44    1
45    1
46    1
47    1
48    1
49    1
50    1
Name: 1, dtype: object

In [77]:
euromillones.boolean_df_especial

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
1,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,True
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1622,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1623,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1624,False,False,False,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
1625,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [18]:
pd.DataFrame({draw: {key: [] for key in euromillones.numbers} for draw in range(1,len(euromillones.boolean_df))}).T

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
1,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
2,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
3,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
4,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
5,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1622,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
1623,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
1624,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
1625,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]


In [14]:
euromillones.counts

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
1,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
2,2,2,2,2,2,2,0,2,2,2,...,1,2,2,2,2,2,0,2,2,0
3,3,3,3,3,3,3,1,3,3,3,...,2,3,3,3,3,3,1,3,3,1
4,4,4,4,0,4,4,0,4,4,4,...,3,4,4,4,4,4,2,4,4,2
5,5,5,5,1,5,5,1,5,5,5,...,4,5,5,0,5,5,0,5,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1622,5,17,10,24,0,10,12,10,5,1,...,13,12,15,3,14,0,2,8,6,1
1623,6,18,11,25,1,11,13,11,6,0,...,14,13,16,4,15,1,3,9,7,2
1624,7,19,12,0,2,12,14,12,7,1,...,15,14,17,0,16,2,4,10,0,3
1625,8,20,13,1,3,13,15,13,8,2,...,16,15,18,1,17,3,5,11,1,4


In [ ]:
current_year = euromillones.lottery.iloc[-1]
year_criteria = {key: [] for key in numbers}

for number in numbers:
    x = euromillones.new_lottery.at[current_year, number]
    median_half = euromillones.median.iat['median',number] / 2

    if euromillones.mean.iat['average',number] != 0 and not np.isnan(
        euromillones.mean.iat['average',number]
        ):
        if x == 0 or x <= median_half:
            y = round((1 - ((euromillones.median.iat['median',number] * 100) / 
                euromillones.mean.iat['average',number]) / 100), 2)
        else:
            x_percentage = round(
                (x * 100 / euromillones.mean.iat['average',number]) / 100, 2
                )
            y = round((1 - x_percentage if x_percentage > 1 else 1 - x_percentage), 2)
    elif y == 0:
        y = 0.50
    else:
        y = 0

    year_criteria[number] = float(y)

year_criteria = pd.DataFrame.from_dict(year_criteria, orient='index', columns=['year_criteria'])

In [ ]:
# Finding missing numbers
best_numbers = df_aprox.loc[df_aprox['Hits'] > df_aprox['Hits_Needed']].index.to_numpy()
normal_numbers = df_exact.loc[(df_exact['Hits'] > df_exact['Hits_Needed']) & ~(df_exact.index.isin(best_numbers))].index.to_numpy()

# Rotation information
rotation_info = {number: 0 for number in total_numbers}
for number in best_numbers:
    rotation_info[number] = 2
for number in normal_numbers:
    rotation_info[number] = 1

# Finding current hits needed
current_hits_needed = data_functions.minimal_hits(db, total_hits, total_numbers, numbers_average, aprox=True)

# Finding rotation hit
rotation_hit = next((draw for draw in range(len(db['Sorteo'])+1, len(db['Sorteo'])+12) if draw * data_functions.minimal_hits(db, total_hits, total_numbers, numbers_average, aprox=True) / len(db) > int(current_hits_needed) + 1), None)

# Rotation criteria
rotation_criteria = {}
for i in total_numbers:
    category = rotation_info[i]
    diff_exact = df_exact.at[i, 'Difference']
    diff_aprox = df_aprox.at[i, 'Difference']
    hits_exact = df_exact.at[i, 'Hits']
    if category == 1 and diff_exact > 0.50:
        rotation_criteria[i] = 0.50
    elif category == 1 and -1 < diff_exact <= 0.40:
        x = (abs(diff_exact) + (Decimal('0.5') * Decimal('100')) / Decimal('0.50')) / Decimal('100') + Decimal('0.50')
        rotation_criteria[i] = float(x.quantize(Decimal('0.01'), rounding=ROUND_HALF_UP))
    elif category == 2 and hits_exact > current_hits_needed:
        rotation_criteria[i] = 1
    elif category == 2 and diff_aprox > -1 and rotation_hit is not None and rotation_hit - len(db) < 4 and len(best_numbers) < 17:
        x = (abs(diff_exact) * Decimal('100')) / Decimal('1.00') / Decimal('100') + Decimal('1')
        rotation_criteria[i] = float(x.quantize(Decimal('0.01'), rounding=ROUND_HALF_UP))
    else:
        rotation_criteria[i] = 0.25

rotation_criteria = pd.DataFrame.from_dict(rotation_criteria, orient='index', columns=['rotation_criteria'])